In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support

In [3]:
#read the pickle files
not_aug_df=pd.read_pickle('EMOVO_dataset/not_aug_df.pkl')
semi_aug_df=pd.read_pickle('EMOVO_dataset/semi_aug_df.pkl')
aug_df=pd.read_pickle('EMOVO_dataset/aug_df.pkl')

In [4]:
test_df = not_aug_df.loc[not_aug_df["actor"] == "f1"]
Y_test = test_df["label_id"]

In [5]:
actors = not_aug_df["actor"].unique()

In [6]:
#create the  stratified k-fold generator of test and training data
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
generator = StratifiedShuffleSplit(n_splits=10)

In [25]:
from datetime import datetime  
name = datetime.now().strftime("ser_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=10,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


#CNN creation
def create_cnn(input_data):
  modelcnn = keras.Sequential()
  modelcnn.add(keras.layers.Input(shape=input_data))
  modelcnn.add(keras.layers.Conv2D(32, (2, 6), activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7), strides=(2,2),padding='same'))

  modelcnn.add(keras.layers.Conv2D(64, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2,7),strides=(2,2), padding='same'))

#   modelcnn.add(keras.layers.Conv2D(128, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
#   modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))

#   modelcnn.add(keras.layers.Conv2D(256, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
#   modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))

 
  modelcnn.add(keras.layers.GlobalAveragePooling2D())
#   modelcnn.add(keras.layers.Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))

  modelcnn.add(keras.layers.Dense(64, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))

  modelcnn.add(keras.layers.Dense(7, activation='softmax'))
  
  return modelcnn

def compile_model(X_logmel):
    input_data=(X_logmel.shape[1:])
    modelcnn=create_cnn(input_data)
    modelcnn.compile(
    optimizer = "Adam", 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
    )
    return modelcnn
def train_val_test(X,Y):
    scaler = StandardScaler()
    X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=22)
    return X_train,X_val,y_train,y_val

def get_accuracy(X_test,y_test, modelcnn):
    predicted = modelcnn.predict(X_test)

    preds = []
    for prediction in predicted:
        preds.append(np.argmax(prediction))

    print(accuracy_score(y_test,preds))


In [26]:
histories_cnn = []
confusion_matrices_cnn=[]
results_cnn=[]
actual_labels_cnn=np.array([])
predicted_labels_cnn=np.array([])

for actor in tqdm(actors):
    scaler = StandardScaler()
    not_aug_df=pd.read_pickle('EMOVO_dataset/not_aug_df.pkl')
    test_df = not_aug_df.loc[not_aug_df["actor"] == actor]
    y_test = test_df["label_id"]

    X_logmel_test = np.array(test_df.loc[:, ['logmel' in i for i in test_df.columns]])
    X_logmel_test=np.array(X_logmel_test.tolist())
    X_test = np.reshape(X_logmel_test, (X_logmel_test.shape[0],X_logmel_test.shape[1],X_logmel_test.shape[2],1))
    X_logmel_test = scaler.fit_transform(X_logmel_test.reshape(-1, X_logmel_test.shape[-1])).reshape(X_logmel_test.shape)


    # X_logmel_k = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
    # X_logmel_k=np.array(X_logmel_k.tolist())
    # X_logmel_k=scaler.fit_transform(X_logmel_k.reshape(-1, X_logmel_k.shape[-1])).reshape(X_logmel_k.shape)
    # Y_logmel_k=not_aug_df['label']

    Y_not_aug=not_aug_df['label_id']
    X_logmel = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
    X_logmel=np.array(X_logmel.tolist())
    X_logmel_semi_aug = np.array(semi_aug_df.loc[:, ['logmel' in i for i in semi_aug_df.columns]])
    X_logmel_semi_aug=np.array(X_logmel_semi_aug.tolist())
    X_logmel_aug = np.array(aug_df.loc[:, ['logmel' in i for i in aug_df.columns]])
    X_logmel_aug=np.array(X_logmel_aug.tolist())
    X_logmel = np.reshape(X_logmel, (X_logmel.shape[0],X_logmel.shape[1],X_logmel.shape[2],1))


    X_train,X_val,y_train,y_val = train_val_test(X_logmel, Y_not_aug)
    
    modelcnn = compile_model(X_logmel)

    history = modelcnn.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=256,epochs=1000, callbacks=callbacks)
    histories_cnn.append(history)
    print(f"Loss : {modelcnn.evaluate(X_test,y_test)[0]}, Accuracy : {modelcnn.evaluate(X_test,y_test)[1]}")
    results = modelcnn.evaluate(X_test, y_test, batch_size=128)
    results_cnn.append(results)


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.1574 - loss: 2.0623
Epoch 1: val_loss improved from inf to 2.04555, saving model to ser_19_09_2024_12_08_47.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 309ms/step - accuracy: 0.1617 - loss: 2.0611 - val_accuracy: 0.2034 - val_loss: 2.0455
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.1702 - loss: 2.0367
Epoch 2: val_loss improved from 2.04555 to 2.04136, saving model to ser_19_09_2024_12_08_47.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.1709 - loss: 2.0361 - val_accuracy: 0.2203 - val_loss: 2.0414
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.1760 - loss: 2.0270
Epoch 3: val_loss improved from 2.04136 to 2.01699, saving model to ser_19_09_2024_12_08_47.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.1748 - loss: 2.0257 - val_accuracy: 0.2119 - val_loss: 2.0170
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.1776 - loss: 2.0109
Epoch 4: val_loss imp

 17%|█▋        | 1/6 [01:05<05:25, 65.16s/it]

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.1554 - loss: 2.0501
Epoch 1: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 304ms/step - accuracy: 0.1604 - loss: 2.0495 - val_accuracy: 0.2288 - val_loss: 2.0388
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.2135 - loss: 2.0310
Epoch 2: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.2104 - loss: 2.0307 - val_accuracy: 0.2203 - val_loss: 2.0218
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.2266 - loss: 2.0128
Epoch 3: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.2227 - loss: 2.0138 - val_accuracy: 0.2373 - val_loss: 2.0100
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.2213 - loss: 2.0038
Epoch 4: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.2234 - loss: 2.0027 - val_accuracy: 0.2119 - val_loss: 1.9960
Epoch 5/

 33%|███▎      | 2/6 [01:11<02:02, 30.56s/it]

...

 67%|██████▋   | 4/6 [01:23<00:28, 14.12s/it]

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.1689 - loss: 2.0701
Epoch 1: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 352ms/step - accuracy: 0.1679 - loss: 2.0671 - val_accuracy: 0.1610 - val_loss: 2.0418
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.1781 - loss: 2.0339
Epoch 2: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.1776 - loss: 2.0338 - val_accuracy: 0.2034 - val_loss: 2.0258
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.1577 - loss: 2.0258
Epoch 3: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - accuracy: 0.1647 - loss: 2.0245 - val_accuracy: 0.2373 - val_loss: 2.0116
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.2308 - loss: 2.0071
Epoch 4: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.2284 - loss: 2.0081 - val_accuracy: 0.2373 - val_loss: 1.9956
Epoch 5/

 83%|████████▎ | 5/6 [01:29<00:11, 11.19s/it]

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.1325 - loss: 2.0636
Epoch 1: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 309ms/step - accuracy: 0.1337 - loss: 2.0626 - val_accuracy: 0.1610 - val_loss: 2.0386
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.1914 - loss: 2.0435
Epoch 2: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - accuracy: 0.1915 - loss: 2.0429 - val_accuracy: 0.2288 - val_loss: 2.0302
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.2277 - loss: 2.0281
Epoch 3: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.2241 - loss: 2.0282 - val_accuracy: 0.1864 - val_loss: 2.0145
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.1925 - loss: 2.0168
Epoch 4: val_loss did not improve from 1.64828
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.1929 - loss: 2.0161 - val_accuracy: 0.2203 - val_loss: 2.0001
Epoch 5/

100%|██████████| 6/6 [01:35<00:00, 15.97s/it]


In [27]:
np.mean(np.array(results_cnn)[:,1])

0.1462585081656774

In [22]:
histories_cnn = []
confusion_matrices_cnn=[]
results_cnn=[]
actual_labels_cnn=np.array([])
predicted_labels_cnn=np.array([])

for actor in tqdm(actors):
    scaler = StandardScaler()
    not_aug_df=pd.read_pickle('EMOVO_dataset/semi_aug_df.pkl')
    test_df = not_aug_df.loc[not_aug_df["actor"] == actor]
    y_test = test_df["label_id"]

    X_logmel_test = np.array(test_df.loc[:, ['logmel' in i for i in test_df.columns]])
    X_logmel_test=np.array(X_logmel_test.tolist())
    X_test = np.reshape(X_logmel_test, (X_logmel_test.shape[0],X_logmel_test.shape[1],X_logmel_test.shape[2],1))
    X_logmel_test = scaler.fit_transform(X_logmel_test.reshape(-1, X_logmel_test.shape[-1])).reshape(X_logmel_test.shape)


    # X_logmel_k = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
    # X_logmel_k=np.array(X_logmel_k.tolist())
    # X_logmel_k=scaler.fit_transform(X_logmel_k.reshape(-1, X_logmel_k.shape[-1])).reshape(X_logmel_k.shape)
    # Y_logmel_k=not_aug_df['label']

    Y_not_aug=not_aug_df['label_id']
    X_logmel = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
    X_logmel=np.array(X_logmel.tolist())
    X_logmel_semi_aug = np.array(semi_aug_df.loc[:, ['logmel' in i for i in semi_aug_df.columns]])
    X_logmel_semi_aug=np.array(X_logmel_semi_aug.tolist())
    X_logmel_aug = np.array(aug_df.loc[:, ['logmel' in i for i in aug_df.columns]])
    X_logmel_aug=np.array(X_logmel_aug.tolist())
    X_logmel = np.reshape(X_logmel, (X_logmel.shape[0],X_logmel.shape[1],X_logmel.shape[2],1))


    X_train,X_val,y_train,y_val = train_val_test(X_logmel, Y_not_aug)
    
    modelcnn = compile_model(X_logmel)

    history = modelcnn.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=256,epochs=1000, callbacks=callbacks)
    histories_cnn.append(history)
    print(f"Loss : {modelcnn.evaluate(X_test,y_test)[0]}, Accuracy : {modelcnn.evaluate(X_test,y_test)[1]}")
    results = modelcnn.evaluate(X_test, y_test, batch_size=128)
    results_cnn.append(results)
    


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.1603 - loss: 2.4753
Epoch 1: val_loss did not improve from 1.68580
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 300ms/step - accuracy: 0.1634 - loss: 2.4713 - val_accuracy: 0.2295 - val_loss: 2.3551
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - accuracy: 0.2458 - loss: 2.3246
Epoch 2: val_loss did not improve from 1.68580
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 277ms/step - accuracy: 0.2452 - loss: 2.3234 - val_accuracy: 0.2663 - val_loss: 2.3016
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - accuracy: 0.2531 - loss: 2.2772
Epoch 3: val_loss did not improve from 1.68580
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 280ms/step - accuracy: 0.2531 - loss: 2.2747 - val_accuracy: 0.2691 - val_loss: 2.2169
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.2772 - loss: 2.1858
Epoch 4: val_loss did not improve from 1.68580
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 286ms/step - accuracy: 0.2767 - loss: 2.1838 - val_accuracy: 0.2408 - val_loss: 2.1823
Epoch 5/

 17%|█▋        | 1/6 [05:38<28:14, 338.80s/it]

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - accuracy: 0.1695 - loss: 2.4806
Epoch 1: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 352ms/step - accuracy: 0.1714 - loss: 2.4776 - val_accuracy: 0.1785 - val_loss: 2.3614
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.2611 - loss: 2.3290
Epoch 2: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 398ms/step - accuracy: 0.2601 - loss: 2.3275 - val_accuracy: 0.2181 - val_loss: 2.3015
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 0.2476 - loss: 2.2520
Epoch 3: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 381ms/step - accuracy: 0.2475 - loss: 2.2506 - val_accuracy: 0.2550 - val_loss: 2.2132
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.2643 - loss: 2.1896
Epoch 4: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 403ms/step - accuracy: 0.2635 - loss: 2.1890 - val_accuracy: 0.2691 - val_loss: 2.1716
Epoch 5/

 33%|███▎      | 2/6 [06:29<11:16, 169.08s/it]

...

 67%|██████▋   | 4/6 [08:09<02:58, 89.12s/it] 

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step - accuracy: 0.1646 - loss: 2.4803
Epoch 1: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 319ms/step - accuracy: 0.1666 - loss: 2.4764 - val_accuracy: 0.2181 - val_loss: 2.3619
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step - accuracy: 0.2445 - loss: 2.3327
Epoch 2: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 390ms/step - accuracy: 0.2450 - loss: 2.3310 - val_accuracy: 0.2295 - val_loss: 2.2666
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.2574 - loss: 2.2313
Epoch 3: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 383ms/step - accuracy: 0.2573 - loss: 2.2309 - val_accuracy: 0.2153 - val_loss: 2.2788
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.2559 - loss: 2.2002
Epoch 4: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 406ms/step - accuracy: 0.2577 - loss: 2.1963 - val_accuracy: 0.2833 - val_loss: 2.1443
Epoch 5/

 83%|████████▎ | 5/6 [08:57<01:14, 74.45s/it]

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step - accuracy: 0.1520 - loss: 2.4732
Epoch 1: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 311ms/step - accuracy: 0.1559 - loss: 2.4688 - val_accuracy: 0.1898 - val_loss: 2.3594
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step - accuracy: 0.2417 - loss: 2.3334
Epoch 2: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 385ms/step - accuracy: 0.2428 - loss: 2.3310 - val_accuracy: 0.2181 - val_loss: 2.2726
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step - accuracy: 0.2475 - loss: 2.2413
Epoch 3: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 385ms/step - accuracy: 0.2477 - loss: 2.2397 - val_accuracy: 0.1870 - val_loss: 2.2039
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step - accuracy: 0.2493 - loss: 2.1618
Epoch 4: val_loss did not improve from 0.47757
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 386ms/step - accuracy: 0.2491 - loss: 2.1614 - val_accuracy: 0.2238 - val_loss: 2.1543
Epoch 5/

100%|██████████| 6/6 [09:45<00:00, 97.59s/it]


In [24]:
results_cnn

[[695.7882080078125, 0.1428571492433548],
 [19.646921157836914, 0.1428571492433548],
 [24.995868682861328, 0.1428571492433548],
 [23.162092208862305, 0.1428571492433548],
 [28.059871673583984, 0.1428571492433548],
 [47.78757858276367, 0.1428571492433548]]

In [23]:
np.mean(np.array(results_cnn)[:,1])

0.1428571492433548

In [28]:
import keras
from keras import layers, models
def get_model():
    inputs = layers.Input(shape=(60,130))
    encoder = layers.LSTM(128)(inputs)
    drop = layers.Dropout(0.3)(encoder)
    hidden = layers.Dense(32, activation='relu')(drop)
    outputs = layers.Dense(7, activation='softmax')(hidden)
    
    model = models.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=["accuracy"])

    return model

In [29]:
X_test.shape

(98, 60, 130, 1)

In [30]:
histories_cnn = []
confusion_matrices_cnn=[]
results_cnn=[]
actual_labels_cnn=np.array([])
predicted_labels_cnn=np.array([])

for actor in tqdm(actors):
    scaler = StandardScaler()
    not_aug_df=pd.read_pickle('EMOVO_dataset/semi_aug_df.pkl')
    test_df = not_aug_df.loc[not_aug_df["actor"] == actor]
    y_test = test_df["label_id"]

    X_logmel_test = np.array(test_df.loc[:, ['logmel' in i for i in test_df.columns]])
    X_logmel_test=np.array(X_logmel_test.tolist())
    X_test = np.reshape(X_logmel_test, (X_logmel_test.shape[0],X_logmel_test.shape[1],X_logmel_test.shape[2],1))
    X_logmel_test = scaler.fit_transform(X_logmel_test.reshape(-1, X_logmel_test.shape[-1])).reshape(X_logmel_test.shape)

    X_test = X_test.squeeze(3)


    # X_logmel_k = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
    # X_logmel_k=np.array(X_logmel_k.tolist())
    # X_logmel_k=scaler.fit_transform(X_logmel_k.reshape(-1, X_logmel_k.shape[-1])).reshape(X_logmel_k.shape)
    # Y_logmel_k=not_aug_df['label']

    Y_not_aug=not_aug_df['label_id']
    X_logmel = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
    X_logmel=np.array(X_logmel.tolist())
    X_logmel_semi_aug = np.array(semi_aug_df.loc[:, ['logmel' in i for i in semi_aug_df.columns]])
    X_logmel_semi_aug=np.array(X_logmel_semi_aug.tolist())
    X_logmel_aug = np.array(aug_df.loc[:, ['logmel' in i for i in aug_df.columns]])
    X_logmel_aug=np.array(X_logmel_aug.tolist())
    X_logmel = np.reshape(X_logmel, (X_logmel.shape[0],X_logmel.shape[1],X_logmel.shape[2],1))


    X_train,X_val,y_train,y_val, = train_val_test(X_logmel.squeeze(3),Y_not_aug)
    LSTM_model = get_model()

    history = LSTM_model.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=256,epochs=1000, callbacks=callbacks)
    histories_cnn.append(history)
    print(f"Loss : {LSTM_model.evaluate(X_test,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test,y_test)[1]}")
    results = LSTM_model.evaluate(X_test, y_test, batch_size=128)
    results_cnn.append(results)
    


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.1419 - loss: 1.9940 
Epoch 1: val_loss did not improve from 1.64828
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 0.1457 - loss: 1.9885 - val_accuracy: 0.1926 - val_loss: 1.9090
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.2415 - loss: 1.8632
Epoch 2: val_loss did not improve from 1.64828
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.2435 - loss: 1.8614 - val_accuracy: 0.2380 - val_loss: 1.8601
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.2642 - loss: 1.7958
Epoch 3: val_loss did not improve from 1.64828
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.2662 - loss: 1.7941 - val_accuracy: 0.2805 - val_loss: 1.8105
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.3032 - loss: 1.7411
Epoch 4: val_loss did not improve from 1.64828
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.3037 - loss: 1.7402 - val_accuracy: 0.3003 - val_loss: 1.7685
Epoch 5/100

 17%|█▋        | 1/6 [00:41<03:26, 41.22s/it]

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.1505 - loss: 1.9793 
Epoch 1: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.1530 - loss: 1.9745 - val_accuracy: 0.1756 - val_loss: 1.9187
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.2551 - loss: 1.8445
Epoch 2: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.2540 - loss: 1.8449 - val_accuracy: 0.2125 - val_loss: 1.8823
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.2970 - loss: 1.7759 
Epoch 3: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.2981 - loss: 1.7736 - val_accuracy: 0.2578 - val_loss: 1.8192
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.3568 - loss: 1.7083
Epoch 4: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.3569 - loss: 1.7068 - val_accuracy: 0.3088 - val_loss: 1.7720
Epoch 5/

 33%|███▎      | 2/6 [00:51<01:32, 23.21s/it]

...

 67%|██████▋   | 4/6 [01:10<00:27, 13.69s/it]

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.1350 - loss: 2.0050
Epoch 1: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.1365 - loss: 2.0007 - val_accuracy: 0.2295 - val_loss: 1.8966
Epoch 2/1000
5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.2248 - loss: 1.8977
Epoch 2: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.2268 - loss: 1.8926 - val_accuracy: 0.2748 - val_loss: 1.8661
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2664 - loss: 1.8332
Epoch 3: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.2677 - loss: 1.8320 - val_accuracy: 0.3003 - val_loss: 1.8243
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.2924 - loss: 1.7961
Epoch 4: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.2945 - loss: 1.7935 - val_accuracy: 0.3371 - val_loss: 1.7851
Epoch 5/1000
6

 83%|████████▎ | 5/6 [01:18<00:11, 11.67s/it]

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.1385 - loss: 2.0124
Epoch 1: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.1409 - loss: 2.0083 - val_accuracy: 0.1870 - val_loss: 1.9124
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.2229 - loss: 1.8871
Epoch 2: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.2235 - loss: 1.8865 - val_accuracy: 0.2408 - val_loss: 1.8738
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2663 - loss: 1.8197
Epoch 3: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.2669 - loss: 1.8198 - val_accuracy: 0.2606 - val_loss: 1.8387
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.3211 - loss: 1.7675
Epoch 4: val_loss did not improve from 0.23735
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.3213 - loss: 1.7661 - val_accuracy: 0.3031 - val_loss: 1.7889
Epoch 5/1000
5

100%|██████████| 6/6 [01:26<00:00, 14.43s/it]


In [31]:
np.mean(np.array(results_cnn)[:,1])

0.15532879903912544